## Импорт библиотек

In [1]:
import pandas as pd
import requests, json
import ast
import datetime as dt

In [2]:
symbol = 'USDTRUB'
start = '1672531200000' # GMT 1 января 2023 года 00:00:00
end = dt.datetime.now() # по настоящее время
end = (end - dt.datetime(1970, 1, 1)) // dt.timedelta(milliseconds=1)
limit = '500'
interval = '15m' 

In [3]:
column_list = ['Kline open time','Open price','High price','Low price','Close price',
                            'Volume','Kline Close time','Quote asset volume','Number of trades',
                            'Taker buy base asset volume','Taker buy quote asset volume','Unused field, ignore']

In [4]:
# создадим интерполяционную сетку, чтоб проще было отсчитывать временные отрезки
ti = pd.date_range(start = pd.to_datetime(start, unit='ms'), end = pd.to_datetime(end, unit='ms'), freq="15min") 
ts = pd.DataFrame({'date':list(ti)})
ts['unixtime'] = (ts.date - pd.Timestamp("1970-01-01 00:00:00")) // pd.Timedelta('1ms')

In [5]:
df = pd.DataFrame(columns=column_list)
start_point = 0
end_point = 0
iteration = 0
attempt = 0
data_amount = len(ts)

while end_point < data_amount-1:
    end_point = start_point+500
    if end_point > data_amount:
        end_point = data_amount-1
    startTime = ts.loc[start_point, 'unixtime']
    endTime = ts.loc[end_point, 'unixtime']
    while attempt < 4:
        try:
            url = f'https://api.binance.com/api/v3/klines?startTime={startTime}&endTime={endTime}&limit={limit}&symbol={symbol}&interval={interval}'
            response = requests.get(url)
            data_list = response.content.decode("utf-8")[2:-2]
            data = data_list.split('],[')
            data = [list(ast.literal_eval(x)) for x in data]
            break
        except:
            attempt += 1
            print(f':start_point {start_point}, end_point: {end_point}.')
            print(f'Время ожидания ответа превышено. {attempt}-я попытка')
            time.sleep(1)
    current_df = pd.DataFrame(data, columns=column_list)
    # Запись текущего датасета в файл
    current_df.to_csv(f"df_{iteration}.csv", index=False)
    df = df.append(current_df, ignore_index=True,
                   verify_integrity=False, sort=False)
    iteration += 1
    start_point = end_point

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10303 entries, 0 to 10302
Data columns (total 12 columns):
 #   Column                        Non-Null Count  Dtype 
---  ------                        --------------  ----- 
 0   Kline open time               10303 non-null  object
 1   Open price                    10303 non-null  object
 2   High price                    10303 non-null  object
 3   Low price                     10303 non-null  object
 4   Close price                   10303 non-null  object
 5   Volume                        10303 non-null  object
 6   Kline Close time              10303 non-null  object
 7   Quote asset volume            10303 non-null  object
 8   Number of trades              10303 non-null  object
 9   Taker buy base asset volume   10303 non-null  object
 10  Taker buy quote asset volume  10303 non-null  object
 11  Unused field, ignore          10303 non-null  object
dtypes: object(12)
memory usage: 966.0+ KB


In [7]:
df.insert(1, 'time', pd.to_datetime(df['Kline open time'], unit='ms')) 
df.drop(['Kline Close time', 'Unused field, ignore'], axis = 1, inplace=True)

In [8]:
df

,Kline open time,time,Open price,High price,Low price,Close price,Volume,Quote asset volume,Number of trades,Taker buy base asset volume,Taker buy quote asset volume
0,1672531200000,2023-01-01 00:00:00,72.38000000,72.39000000,72.36000000,72.39000000,782.00000000,56602.48000000,17,560.00000000,40536.27000000
1,1672532100000,2023-01-01 00:15:00,72.39000000,72.39000000,72.37000000,72.37000000,3706.00000000,268254.08000000,22,2484.00000000,179809.76000000
2,1672533000000,2023-01-01 00:30:00,72.39000000,72.39000000,72.38000000,72.39000000,1197.00000000,86649.00000000,19,1014.00000000,73403.46000000
3,1672533900000,2023-01-01 00:45:00,72.39000000,72.39000000,72.38000000,72.38000000,818.00000000,59212.19000000,12,535.00000000,38728.65000000
4,1672534800000,2023-01-01 01:00:00,72.38000000,72.39000000,72.38000000,72.38000000,182.00000000,13173.59000000,11,43.00000000,3112.77000000
...,...,...,...,...,...,...,...,...,...,...,...
10298,1681803000000,2023-04-18 07:30:00,82.95000000,82.97000000,82.89000000,82.95000000,12815.00000000,1062717.98000000,122,4909.00000000,407180.50000000
10299,1681803900000,2023-04-18 07:45:00,82.95000000,82.96000000,82.89000000,82.95000000,8219.00000000,681575.25000000,85,5238.00000000,434408.75000000
10300,1681804800000,2023-04-18 08:00:00,82.96000000,82.98000000,82.94000000,82.96000000,8680.00000000,720101.12000000,84,4406.00000000,365539.05000000
10301,1681805700000,2023-04-18 08:15:00,82.96000000,83.00000000,82.89000000,82.91000000,17073.00000000,1416271.32000000,140,8670.00000000,719284.73000000


In [9]:
df.to_csv('df_for_EDA', index=False)